# Ph.D. Course - Data Vizualization 
#### Authors
-  Manuele Pasini - manuele.pasini@unibo.it
- Gianni Tumedei - gianni.tumedei2@unibo.it

In [15]:
import pandas as pd
import os
import numpy as np 

input_path = os.path.join(os.sep, "home", "dataset")
output_path = os.path.join(os.sep, "home", "results")

In [ ]:
def describe_dataframe(df):
    summary = []
    for col in df.columns:
        dtype = df[col].dtype
        null_count = df[col].isnull().sum()
        null_pct = 100 * null_count / len(df)

        stats = {
            "colonna": col,
            "tipo": str(dtype),
            "nulli": null_count,
            "% nulli": round(null_pct, 2)
        }

        if pd.api.types.is_numeric_dtype(df[col]):
            stats.update({
                "count": df[col].count(),
                "mean": df[col].mean(),
                "std": df[col].std(),
                "min": df[col].min(),
                "max": df[col].max()
            })
        summary.append(stats)

    return pd.DataFrame(summary)

#### Preprocessing

In [ ]:
devices = pd.read_csv(os.path.join(input_path, 'devices.csv'))
measurements = pd.read_csv(os.path.join(input_path, 'measurements.csv'))

measurements["deveui"] = measurements["deveui"].str.replace("-", "").str.upper()
measurements = measurements.drop(columns=["acceleration_x", "acceleration_y", "acceleration_z"])
measurements = measurements.rename(columns={"deveui": "device_id"})

devices = devices.rename(columns={"id": "device_id"})
devices = devices.drop(columns=["type", "status", "installed", "addedBy"])

# Merge the two dataframes on the 'device_id' column
merged_df = pd.merge(devices, measurements, on='device_id')

# Convert the 'location' column to a (lon,lat) format
merged_df["location"] = merged_df.apply(lambda row: f"{row['location/0']}, {row['location/1']}", axis=1)
merged_df = merged_df.drop(columns=["location/0", "location/1"])

# Convert the 'timestamp' column to datetime format
merged_df["time"] = pd.to_datetime(merged_df["time"], unit='ms')
merged_df["time"] = merged_df["time"].dt.strftime('%Y-%m-%d %H:%M:%S')

# Remove columns with more than 60% missing values
missing_threshold = 0.6
cleaned_df = merged_df.loc[:, merged_df.isnull().mean() < missing_threshold]

merged_df.to_csv(os.path.join(output_path, 'merged_data.csv'), index=False)

,floor,temperature,humidity,light,motion,co2,battery,sound_avg,sound_peak,moisture,pressure
count,6.152351e+06,6.149539e+06,6.149539e+06,6.037914e+06,6.086420e+06,4.241850e+06,6.152351e+06,1.729062e+06,1.729062e+06,114437.000000,114437.000000
mean,1.464001e+00,2.045445e+01,3.205708e+01,6.277592e+01,2.228724e+01,6.532731e+02,3.668384e+00,3.728909e+01,6.581642e+01,477.695588,51.821642
std,9.769463e-01,4.143864e+00,1.655875e+01,1.611226e+02,3.454671e+01,3.509550e+03,3.374035e-02,6.340620e+00,4.411980e+00,1321.051543,464.301952
min,-1.000000e+00,3.600000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.726000e+00,3.400000e+01,6.400000e+01,0.000000,0.971100
25%,1.000000e+00,1.960000e+01,2.000000e+01,1.000000e+00,0.000000e+00,4.080000e+02,3.639000e+00,3.400000e+01,6.400000e+01,21.000000,1.008700
50%,1.000000e+00,2.060000e+01,3.100000e+01,7.000000e+00,0.000000e+00,4.560000e+02,3.678000e+00,3.400000e+01,6.400000e+01,31.000000,1.018700
75%,2.000000e+00,2.150000e+01,4.300000e+01,6.900000e+01,5.500000e+01,5.070000e+02,3.690000e+00,3.800000e+01,6.400000e+01,36.000000,1.025700
max,5.000000e+00,6.508600e+03,1.000000e+02,2.381000e+03,2.530000e+02,6.553400e+04,3.778000e+00,7.000000e+01,9.900000e+01,4369.000000,4294.967300


In [34]:
print(f"Number of devices: {merged_df["device_id"].unique().size}")
print(f"Number of distinct devices {merged_df["desc"].unique().size}")

print(f"Number of measurements: {merged_df.size}")
print(f"Measurements time window: from {merged_df["time"].min()} to {merged_df["time"].max()}")

describe_dataframe(merged_df)

merged_df

Number of devices: 429
Number of distinct devices 358
Number of measurements: 92285265
Measurements time window: from 2020-08-01 00:00:02 to 2021-02-23 20:32:28


,device_id,floor,desc,time,temperature,humidity,light,motion,co2,battery,sound_avg,sound_peak,moisture,pressure,location
0,A81758FFFE046433,4,Attached to old video projector mount,2020-08-01 00:01:10,23.6,39.0,2.0,20.0,NaN,3.641,34.0,64.0,NaN,NaN,"65.05765, 25.46897"
1,A81758FFFE046433,4,Attached to old video projector mount,2020-08-01 00:05:11,23.7,39.0,2.0,21.0,NaN,3.638,34.0,64.0,NaN,NaN,"65.05765, 25.46897"
2,A81758FFFE046433,4,Attached to old video projector mount,2020-08-01 00:09:10,23.6,39.0,2.0,21.0,NaN,3.638,34.0,64.0,NaN,NaN,"65.05765, 25.46897"
3,A81758FFFE046433,4,Attached to old video projector mount,2020-08-01 00:13:11,23.7,39.0,2.0,22.0,NaN,3.641,34.0,64.0,NaN,NaN,"65.05765, 25.46897"
4,A81758FFFE046433,4,Attached to old video projector mount,2020-08-01 00:17:11,23.6,39.0,2.0,18.0,NaN,3.641,34.0,64.0,NaN,NaN,"65.05765, 25.46897"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6152346,A81758FFFE030FFE,1,Hallway next to LO105,2021-02-23 19:29:38,21.8,7.0,41.0,2.0,635.0,3.689,NaN,NaN,NaN,NaN,"65.05809, 25.46576"
6152347,A81758FFFE030FFE,1,Hallway next to LO105,2021-02-23 19:44:38,21.9,7.0,3.0,0.0,640.0,3.686,NaN,NaN,NaN,NaN,"65.05809, 25.46576"
6152348,A81758FFFE030FFE,1,Hallway next to LO105,2021-02-23 19:59:38,21.8,7.0,3.0,0.0,636.0,3.689,NaN,NaN,NaN,NaN,"65.05809, 25.46576"
6152349,A81758FFFE030FFE,1,Hallway next to LO105,2021-02-23 20:14:38,21.8,7.0,3.0,0.0,637.0,3.686,NaN,NaN,NaN,NaN,"65.05809, 25.46576"


In [35]:
describe_dataframe(merged_df)

,colonna,tipo,nulli,% nulli,count,mean,std,min,max
0,device_id,object,0,0.00,NaN,NaN,NaN,NaN,NaN
1,floor,int64,0,0.00,6152351.0,1.464001,0.976946,-1.0000,5.0000
2,desc,object,0,0.00,NaN,NaN,NaN,NaN,NaN
3,time,object,0,0.00,NaN,NaN,NaN,NaN,NaN
4,temperature,float64,2812,0.05,6149539.0,20.454447,4.143864,3.6000,6508.6000
5,humidity,float64,2812,0.05,6149539.0,32.057076,16.558750,0.0000,100.0000
6,light,float64,114437,1.86,6037914.0,62.775918,161.122607,0.0000,2381.0000
7,motion,float64,65931,1.07,6086420.0,22.287236,34.546710,0.0000,253.0000
8,co2,float64,1910501,31.05,4241850.0,653.273145,3509.549829,1.0000,65534.0000
9,battery,float64,0,0.00,6152351.0,3.668384,0.033740,1.7260,3.7780


358